In [4]:
import math
import re
from collections import defaultdict

In [73]:
directions = ["right", "down", "left", "up"]

class Node:
    def __init__(self, y, x):
        self.x = x
        self.y = y
        self.neighbours = {d: None for d in directions}
        self.face = {d: k for k, d in enumerate(directions)}

class Board:
    def __init__(self, map_instructions):
        self.nodes = defaultdict(dict)
        self.coords = defaultdict(list)
        self.instructions = re.split(r"([LR])", map_instructions[-1])
        mapstrings = map_instructions[:-2]
        n_col = 0
        for j, r in enumerate(mapstrings):
            n_col = max(n_col, len(r))
            for k, c in enumerate(r):
                if c == '.':
                    self.nodes[j][k] = Node(j, k)
                    self.coords[j].append(k)
                    if k in self.coords[j - 1]:
                        self.nodes[j][k].neighbours["up"] = self.nodes[j - 1][k]
                        self.nodes[j-1][k].neighbours["down"] = self.nodes[j][k]
                    if k - 1 in self.coords[j]:
                        self.nodes[j][k].neighbours["left"] = self.nodes[j][k - 1]
                        self.nodes[j][k - 1].neighbours["right"] = self.nodes[j][k]
        # add periodicity (horizontal)
        for j, r in enumerate(mapstrings):
            row = r.strip()
            if row[0] == '.' and row[-1] == '.':
                start = self.nodes[j][self.coords[j][0]]
                end = self.nodes[j][self.coords[j][-1]]
                start.neighbours["left"] = end
                end.neighbours["right"] = start
        # add periodicity (vertical):
        for k in range(n_col):
            first_i = 0
            while len(mapstrings[first_i]) <= k or mapstrings[first_i][k] == ' ':
                first_i += 1
            last_i = len(mapstrings) - 1
            while len(mapstrings[last_i]) <= k or mapstrings[last_i][k] == ' ':
                last_i -= 1
            if mapstrings[first_i][k] == '.' and mapstrings[last_i][k] == '.':
                start = self.nodes[first_i][k]
                end = self.nodes[last_i][k]
                start.neighbours["up"] = end
                end.neighbours["down"] = start
        
        # clean dicts
        self.nodes = {k: v for k, v in self.nodes.items() if v}
        self.coords = {k: v for k, v in self.coords.items() if v}
        
    def path(self):
        location = self.nodes[0][self.coords[0][0]] #first node
        facing = 0
        for i in self.instructions:
            if i == "L":
                facing = (facing - 1) % 4
            elif i == "R":
                facing = (facing + 1) % 4
            else:
                direction = directions[facing]
                for _ in range(int(i)):
                    if location.neighbours[direction] is None:
                        break
                    else:
                        location = location.neighbours[direction]
        password = 1000 * (location.y + 1) + 4 * (location.x + 1) + facing
        return password, location.y, location.x, facing

class Cube:
    def __init__(self, map_instructions, face_coordinates, face_links):
        self.nodes = defaultdict(dict)
        self.coords = defaultdict(list)
        self.instructions = re.split(r"([LR])", map_instructions[-1])
        mapstrings = map_instructions[:-2]
        self.l = math.isqrt(sum(sum(x != ' ' for x in l) for l in mapstrings) // 6)
        n_col = 0
        for j, r in enumerate(mapstrings):
            n_col = max(n_col, len(r))
            for k, c in enumerate(r):
                if c == '.':
                    self.nodes[j][k] = Node(j, k)
                    self.coords[j].append(k)
                    if k in self.coords[j - 1]:
                        self.nodes[j][k].neighbours["up"] = self.nodes[j - 1][k]
                        self.nodes[j-1][k].neighbours["down"] = self.nodes[j][k]
                    if k - 1 in self.coords[j]:
                        self.nodes[j][k].neighbours["left"] = self.nodes[j][k - 1]
                        self.nodes[j][k - 1].neighbours["right"] = self.nodes[j][k]
        for (f_n1, d1), (f_n2, d2) in face_links:
            self.join(face_coordinates[f_n1], face_coordinates[f_n2], d1, d2)
        # clean dicts
        self.nodes == {k: v for k, v in self.nodes.items() if v}
        self.coords == {k: v for k, v in self.coords.items() if v}

        
    def side(self, face, direction):
        if direction == 0:
            return ((face[0] * self.l + k, (face[1] + 1) * self.l - 1) for k in range(self.l))
        if direction == 1:
            return (((face[0] + 1) * self.l - 1, face[1] * self.l + k) for k in range(self.l))
        if direction == 2:
            return ((face[0] * self.l + k, face[1] * self.l) for k in range(self.l))
        if direction == 3:
            return ((face[0] * self.l, face[1] * self.l + k) for k in range(self.l))

    def join(self, face1, face2, dir1, dir2):
        s1, s2 = self.side(face1, dir1), self.side(face2, dir2)
        if dir1 == dir2 or set((dir1, dir2)) == {1, 2} or set((dir1, dir2)) == {0, 3}: #condition on dir1, dir2 to determine whether to reverse
            s2 = (x for x in list(s2)[::-1])
        for (y1, x1), (y2, x2) in zip(s1, s2):
            if x1 in self.coords[y1] and x2 in self.coords[y2]:
                start = self.nodes[y1][x1]
                end = self.nodes[y2][x2]
                start.neighbours[directions[dir1]] = end
                end.neighbours[directions[dir2]] = start
                start.face[directions[dir1]] = (dir2 + 2) % 4
                end.face[directions[dir2]] = (dir1 + 2) % 4
                    
    def path(self):
        location = self.nodes[0][self.coords[0][0]] #first node
        facing = 0
        for i in self.instructions:
            if i == "L":
                facing = (facing - 1) % 4
            elif i == "R":
                facing = (facing + 1) % 4
            else:
                for _ in range(int(i)):
                    direction = directions[facing]
                    if location.neighbours[direction]:
                        facing = location.face[direction]
                        location = location.neighbours[direction]
                    else:
                        continue

                    #print(1000 * (location.y + 1) + 4 * (location.x + 1) + facing, location.y + 1, location.x + 1, facing)
        password = 1000 * (location.y + 1) + 4 * (location.x + 1) + facing
        return password, location.y, location.x, facing


In [74]:
input_file = "22_input.txt"
face_coordinates = {1: (0, 1), 2: (0, 2), 3: (1, 1), 4: (3, 0), 5: (2, 0), 6: (2, 1)}
face_links = (
    ((1, 3), (4, 2)),
    ((1, 2), (5, 2)),
    ((2, 3), (4, 1)),
    ((2, 0), (6, 0)),
    ((2, 1), (3, 0)),
    ((3, 2), (5, 3)),
    ((4, 0), (6, 1)),
)

with open(input_file) as f:
    inputstrings = [line.rstrip('\n') for line in f]


sample_file = "22_sample.txt"
sample_coordinates = {1: (0, 2), 2: (1, 1), 3: (1, 2), 4: (1, 0), 5: (2, 3), 6: (2, 2)}
sample_links = (
    ((1, 0), (5, 0)),
    ((3, 0), (5, 3)),
    ((1, 2), (2, 3)),
    ((1, 3), (4, 3)),
    ((4, 2), (5, 1)),
    ((2, 1), (6, 2)),
    ((4, 1), (6, 1)),
)

    
with open(sample_file) as f:
    samplestrings = [line.rstrip('\n') for line in f]

In [75]:
myboard = Board(samplestrings)
print(myboard.path())
cube = Cube(samplestrings, sample_coordinates, sample_links)
print(cube.path())

(6032, 5, 7, 0)
(5031, 4, 6, 3)


In [76]:
myboard = Board(inputstrings)
myboard.path()

(155060, 154, 14, 0)

In [77]:
cube.nodes[0]

{8: <__main__.Node at 0x114210580>,
 9: <__main__.Node at 0x114210040>,
 10: <__main__.Node at 0x11495a910>}

In [78]:
cube = Cube(inputstrings, face_coordinates, face_links)
cube.path()

(3479, 2, 118, 3)